In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as ticker

In [ ]:
def mel_filter_banks(idx, path):
    y, sr = librosa.load(path + str(idx) + '.mp3', sr=16000)
    
    # 获取0-3秒的部分
    tmax, tmin = 3, 0
    t = np.linspace(tmin, tmax, (tmax - tmin) * sr)
    # personal_plot(t, y[tmin * sr:tmax * sr], idx , path)
    
    alpha = 0.97
    emphasized_y = np.append(y[tmin * sr], y[tmin * sr + 1:tmax * sr] - alpha * y[tmin * sr:tmax * sr - 1])
    n = int((tmax - tmin) * sr)  # 信号一共的sample数量
    
    # 未经过预加重的信号频谱
    # plt.figure(dpi=300, figsize=(7, 4))
    # freq = sr / n * np.linspace(0, n / 2, int(n / 2) + 1)
    # plt.plot(freq, np.absolute(np.fft.rfft(y[tmin * sr:tmax * sr], n) ** 2) / n)
    # plt.xlim(0, 5000)
    # plt.xlabel('Frequency/Hz', fontsize=14)
    # # plt.show()
    # plt.savefig(path + str(idx) +'spectrum.png')
    
    # # 预加重之后的信号频谱
    # plt.figure(dpi=300, figsize=(7, 4))
    # plt.plot(freq,np.absolute(np.fft.rfft(emphasized_y,n)**2)/n)
    # plt.xlim(0, 5000)
    # plt.xlabel('Frequency/Hz', fontsize=14)
    # # plt.show()
    # plt.savefig(path + str(idx) +'spectrum_emphasized.png')
    
    frame_size, frame_stride = 0.025, 0.01
    frame_length, frame_step = int(round(sr*frame_size)), int(round(sr*frame_stride))
    # print(frame_length, frame_step)
    signal_length = (tmax-tmin)*sr
    # 向上舍入
    frame_num = int(np.ceil((signal_length-frame_length)/frame_step))+1
    # 不足的部分补零
    pad_frame = (frame_num-1)*frame_step+frame_length-signal_length
    pad_y = np.append(emphasized_y, np.zeros(pad_frame))
    signal_len = signal_length+pad_frame
    
    indices = np.tile(np.arange(0, frame_length), (frame_num, 1)) + np.tile(
        np.arange(0, frame_num * frame_step, frame_step), (frame_length, 1)).T
    # frame的每一行代表每一帧的sample值
    frames = pad_y[indices]
    # 加hamming window 注意这里不是矩阵乘法
    frames *= np.hamming(frame_length)
    
    # frame_length=1102，所以用1024足够了
    NFFT =1024
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))
    pow_frames = mag_frames ** 2 / NFFT
    
    plt.figure(dpi=300, figsize=(12, 6))
    plt.imshow(20 * np.log10(pow_frames[40:].T), cmap=plt.cm.jet, aspect='auto')
    plt.yticks([0, 128, 256, 384, 512], np.array([0, 128, 256, 384, 512]) * sr / NFFT)
    # plt.show()
    plt.savefig(path + str(idx) + 'spectrogram.png')
    
    # 下面定义mel filter
    # 滤波器数量,这个数字若要提高，则NFFT也要相应提高
    mel_N = 40
    mel_low, mel_high = 0, (2595 * np.log10(1 + (sr / 2) / 700))
    mel_freq = np.linspace(mel_low, mel_high, mel_N + 2)
    hz_freq = (700 * (10 ** (mel_freq / 2595) - 1))
    # 将频率转换成对应的sample位置
    bins = np.floor((NFFT) * hz_freq / sr)
    # 每一行储存一个梅尔滤波器的数据
    fbank = np.zeros((mel_N, int(NFFT / 2 + 1)))
    for m in range(1, mel_N + 1):
        f_m_minus = int(bins[m - 1])  # left
        f_m = int(bins[m])  # center
        f_m_plus = int(bins[m + 1])  # right
    
        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bins[m - 1]) / (bins[m] - bins[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bins[m + 1] - k) / (bins[m + 1] - bins[m])
    filter_banks = np.matmul(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # np.finfo(float)是最小正值
    filter_banks = 20 * np.log10(filter_banks)
    # filter_banks -= np.mean(filter_banks,axis=1).reshape(-1,1)
    print(filter_banks.shape)          # (299, 40)
    print(type(filter_banks))
    filter_banks = filter_banks.flatten()
    print(filter_banks.shape)



    # plt.figure(dpi=300, figsize=(12, 6))
    # plt.imshow(filter_banks[40:].T, cmap=plt.cm.jet, aspect='auto')
    # plt.yticks([0, 10, 20, 30, 39], [0, 1200, 3800, 9900, 22000])
    # # plt.show()
    # plt.savefig(path + 'mel_filter_banks' + str(idx) + '.png')
    # plt.close()

    np.savez(path+str(idx)+'feature.npz', filter_banks)

In [ ]:
if __name__ == "__main__":
    n = 0
    while n < 2447:
        for j in range(n, n+100):
            if j == 2447:
                break
            mel_filter_banks(j, path)
            print(j)
        n += 100

    # mel_filter_banks(2446, path)

    # for i in range(8660, 8680):
    #     mel_filter_banks(i, path)
    #     # librosa_mel_filter(i, path)
    #     print(i)



    print('done')